# Data Modeling - Diabetes Dataset

**Authors:**  
Filip Kobus, Łukasz Jarzęcki, Paweł Skierkowski  
**Date:** 23.01.25  
**Team 3**

## Imports

In [ ]:
import pathlib
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

sys.path.append(str(pathlib.Path.cwd().parent / 'src'))

from xgboost import XGBRegressor

## Preprocess data

In [ ]:
diabetes = pd.read_csv('diabetes_dataset.csv').drop(['diabetes_stage'], axis=1)

NUM_COLS = [
    'age',
    'alcohol_consumption_per_week',
    'physical_activity_minutes_per_week',
    'diet_score',
    'sleep_hours_per_day',
    'screen_time_hours_per_day',
    'bmi',
    'waist_to_hip_ratio',
    'systolic_bp',
    'diastolic_bp',
    'heart_rate',
    'cholesterol_total',
    'hdl_cholesterol',
    'ldl_cholesterol',
    'triglycerides',
    'glucose_fasting',
    'glucose_postprandial',
    'insulin_level',
    'hba1c',
]

CAT_COLS = [
    'family_history_diabetes',
    'hypertension_history',
    'cardiovascular_history',
]

ORDINAL_ENCODER_COLS = ['education_level', 'income_level', 'smoking_status']
ORDINAL_ENCODER_CATS = [
    ['No formal', 'Highschool', 'Graduate', 'Postgraduate'],
    ['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High'],
    ['Never', 'Former', 'Current'],
]
ONEHOT_ENCODER_COLS = ['employment_status', 'gender', 'ethnicity']


def read_data(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path).drop(['diabetes_stage', 'diagnosed_diabetes'], axis=1)

    return df


def build_preprocessor(
    num_columns: list[str],
    cat_columns: list[str],
    ordinal_enc_columns: list[str],
    ordinal_enc_categories: list[list[str]],
    one_hot_enc_columns: list[str],
) -> ColumnTransformer:
    preprocessor = ColumnTransformer(
        [
            (
                'num_pipeline',
                Pipeline(
                    [
                        ('median_imputer', SimpleImputer(strategy='median')),
                        ('scaler', StandardScaler()),
                    ]
                ),
                num_columns,
            ),
            (
                'cat_pipeline',
                Pipeline(
                    [
                        (
                            'most_frequent_imputer',
                            SimpleImputer(strategy='most_frequent'),
                        ),
                    ]
                ),
                cat_columns,
            ),
            (
                'ord_pipeline',
                Pipeline(
                    [
                        (
                            'most_frequent_imputer',
                            SimpleImputer(strategy='most_frequent'),
                        ),
                        (
                            'ordinal_encoder',
                            OrdinalEncoder(categories=ordinal_enc_categories),
                        ),
                    ]
                ),
                ordinal_enc_columns,
            ),
            (
                'onehot_pipeline',
                Pipeline(
                    [
                        (
                            'most_frequent_imputer',
                            SimpleImputer(strategy='most_frequent'),
                        ),
                        (
                            'ordinal_encoder',
                            OneHotEncoder(
                                drop='first',
                                sparse_output=False,
                                handle_unknown='ignore',
                            ),
                        ),
                    ]
                ),
                one_hot_enc_columns,
            ),
        ]
    )

    return preprocessor


def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:
    diabetes = df
    # blood pressure
    impossible_bp = diabetes[
        (diabetes['systolic_bp'] <= diabetes['diastolic_bp'])
        | (diabetes['systolic_bp'] < 60)
        | (diabetes['systolic_bp'] > 300)
    ]

    # cholesterol
    cholesterol_error = diabetes[
        diabetes['cholesterol_total']
        < (diabetes['hdl_cholesterol'] + diabetes['ldl_cholesterol']) - 20
    ]

    # screen time, sleep hours, age
    sleep_error = diabetes[
        (diabetes['sleep_hours_per_day'] <= 0) | (diabetes['sleep_hours_per_day'] > 24)
    ]

    age_error = diabetes[(diabetes['age'] < 0) | (diabetes['age'] > 110)]

    screentime_error = diabetes[
        (diabetes['screen_time_hours_per_day'] < 0)
        | (diabetes['screen_time_hours_per_day'] > 24)
    ]

    # WHR (Waist to hip ratio)
    whr_error = diabetes[
        (diabetes['waist_to_hip_ratio'] < 0.5) | (diabetes['waist_to_hip_ratio'] > 2.5)
    ]

    index_to_drop = (
        set(impossible_bp.index)
        | set(cholesterol_error.index)
        | set(sleep_error.index)
        | set(age_error.index)
        | set(screentime_error.index)
        | set(whr_error.index)
    )

    diabetes_clean = diabetes.drop(index=list(index_to_drop)).copy()

    return diabetes_clean


def split_data(df: pd.DataFrame) -> tuple:
    target = 'diabetes_risk_score'
    test_size = 0.2
    random_state = 42

    y = df[target]
    X = df.drop(target, axis=1)

    return train_test_split(X, y, test_size=test_size, random_state=random_state)


def format_col_names(cols: list[str]) -> list[str]:
    new_cols = []
    for col in cols:
        new_cols.append(col.split('__')[1])
    return new_cols


def save_data(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    X_train.to_csv('X_train.csv', index=False)
    X_test.to_csv('X_test.csv', index=False)
    y_train.to_csv('y_train.csv', index=False)
    y_test.to_csv('y_test.csv', index=False)


def process_features(
    preprocessor: ColumnTransformer, X_train: pd.DataFrame, X_test: pd.DataFrame
) -> tuple[pd.DataFrame, pd.DataFrame]:
    X_train_pre = preprocessor.fit_transform(X_train)
    X_test_pre = preprocessor.transform(X_test)
    feature_names = format_col_names(preprocessor.get_feature_names_out())

    X_train_pre_df = pd.DataFrame(X_train_pre, columns=feature_names)
    X_test_pre_df = pd.DataFrame(X_test_pre, columns=feature_names)

    return X_train_pre_df, X_test_pre_df


def preprocess_data():
    df = read_data('diabetes_dataset.csv')

    df_cleaned = remove_outliers(df)

    y_train: pd.Series
    y_test: pd.Series
    X_train, X_test, y_train, y_test = split_data(df_cleaned)

    preprocessor = build_preprocessor(
        NUM_COLS,
        CAT_COLS,
        ORDINAL_ENCODER_COLS,
        ORDINAL_ENCODER_CATS,
        ONEHOT_ENCODER_COLS,
    )

    X_train_pre_df, X_test_pre_df = process_features(preprocessor, X_train, X_test)

    save_data(X_train_pre_df, X_test_pre_df, y_train, y_test)


if __name__ == '__main__':
    preprocess_data()

The most important data cleaning decisions were the removal of records with medical inconsistencies, such as illogical blood pressure or cholesterol levels, to maintain physiological validity. Furthermore, ordinal encoding was used for hierarchical features to ensure that categorical data was transformed correctly for the modeling phase.

In [ ]:
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()

final_results = []

## Modeling

Before applying linear regression we check how highly corelated features such as: LDL cholesterol, HbA1, and waist-to-hip ratio imply the model.

**EDA cite:**
> Glucose postprandial and HbA1c are extremely highly correlated (r=0.93), as are total cholesterol and LDL cholesterol (r=0.91). Fasting glucose correlates strongly with both HbA1c (r=0.70) and postprandial glucose (r=0.59). BMI and waist-to-hip ratio also show high correlation (r=0.77).

Drop one column for a correlated pair - chosen randomly.

In [ ]:
corelated_features = ['waist_to_hip_ratio', 'ldl_cholesterol', 'hba1c']
X_noncor_train = X_train.drop(columns=corelated_features)
X_noncor_test = X_test.drop(columns=corelated_features)

In [ ]:
# LINEAR REGRESSION
def linear_regression_model(X_train, y_train, X_test, y_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    y_pred_lr = lr_model.predict(X_test)

    rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
    r2_lr = r2_score(y_test, y_pred_lr)

    print(f'Linear Regression RMSE: {rmse_lr:.6f}')
    print(f'Linear Regression R2: {r2_lr:.6f}')

    final_results.append({'Model': 'Linear Regression', 'RMSE': rmse_lr, 'R2': r2_lr})

    return lr_model

Check the RMSE and R2 between two approaches.

In [ ]:
print('Linear Regression with all features:')
lr_before_drop = linear_regression_model(X_train, y_train, X_test, y_test)
print('Linear Regression after dropping correlated features:')
lr_after_drop = linear_regression_model(X_noncor_train, y_train, X_noncor_test, y_test)

Compare coeficients of those two models, I check coefs on features that where the pair for the removed ones.

In [ ]:
features_to_check = ['glucose_postprandial', 'cholesterol_total', 'bmi']
print('Coef before dropping correlated features:')
for feature in features_to_check:
    coef = lr_after_drop.coef_[list(X_noncor_train.columns).index(feature)]
    print(f'{feature}: {coef}')
print('\nCoef after dropping correlated features:')
for feature in features_to_check:
    coef = lr_before_drop.coef_[list(X_train.columns).index(feature)]
    print(f'{feature}: {coef}')

Dropping highly correlated variables (e.g., hba1c, ldl_cholesterol) did not decrease predictive accuracy, confirming that the dataset contained significant information redundancy. However, this step was essential for model interpretability; removing multicollinearity "unmasked" the true influence of key predictors, such as glucose_postprandial, which saw its coefficient increase tenfold.

In [ ]:
lr_model = lr_after_drop
X_train = X_noncor_train
X_test = X_noncor_test

We picked the model trained on the dataset without corelated features as the linear regression model for further analysis.

In [ ]:
# RIDGE
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)

y_pred_ridge = ridge_model.predict(X_test)

rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f'Ridge Regression RMSE: {rmse_ridge:.6f}')
print(f'Ridge Regression R2: {r2_ridge:.6f}')

final_results.append({'Model': 'Ridge', 'RMSE': rmse_ridge, 'R2': r2_ridge})

Ridge regression achieved the same performance as the linear model with an R2 of 0.9935. This shows that the data has a strong linear structure and that L2 regularization successfully maintains accuracy while protecting the model from extreme coefficient values caused by correlation.

In [ ]:
# LASSO
lasso_model = Lasso(alpha=0.1, random_state=42)
lasso_model.fit(X_train, y_train)

y_pred_lasso = lasso_model.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f'Lasso RMSE: {rmse_lasso:.6f}')
print(f'Lasso R2: {r2_lasso:.6f}')
final_results.append({'Model': 'Lasso', 'RMSE': rmse_lasso, 'R2': r2_lasso})

Lasso regression has a slightly higher error than Ridge because it removed redundant features to simplify the model. The performance is still very high (R2 = 0.992), showing that we can identify the most important risk factors without losing accuracy.

In [ ]:
# XGBOOST
xgb_model = XGBRegressor(
    n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1
)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f'XGBoost RMSE: {rmse_xgb:.4f}')
print(f'XGBoost R2: {r2_xgb:.6f}')
final_results.append({'Model': 'XGBoost', 'RMSE': rmse_xgb, 'R2': r2_xgb})

XGBoost performed the best with the lowest RMSE (0.3069) and the highest R2 (0.9988). This shows that tree-based models are better at capturing complex non-linear patterns in the data compared to linear models. XGBoost is also naturally resistant to highly correlated features, allowing it to achieve better accuracy than linear regresion.

In [ ]:
# TUNING
# Ridge
linear_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

ridge_grid = GridSearchCV(
    Ridge(random_state=42), linear_params, scoring='neg_root_mean_squared_error', cv=5
)
ridge_grid.fit(X_train, y_train)

print(f'Best Ridge Alpha: {ridge_grid.best_params_}')
print(f'Best Ridge RMSE: {-ridge_grid.best_score_:.4f}')

# Lasso
lasso_grid = GridSearchCV(
    Lasso(random_state=42), linear_params, scoring='neg_root_mean_squared_error', cv=5
)
lasso_grid.fit(X_train, y_train)

print(f'Best Lasso Alpha: {lasso_grid.best_params_}')
print(f'Best Lasso RMSE: {-lasso_grid.best_score_:.4f}')

best_ridge = ridge_grid.best_estimator_
best_lasso = lasso_grid.best_estimator_

# xgboost
xgb_params = {
    'n_estimators': [500, 1000, 2000],
    'max_depth': [2, 3, 4],
    'min_child_weight': [5, 7, 10, 15],
    'learning_rate': [0.005, 0.01, 0.02, 0.05],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.1, 1, 5],
    'reg_lambda': [1, 2, 5],
}
xgb_search = RandomizedSearchCV(
    XGBRegressor(random_state=42, n_jobs=-1),
    param_distributions=xgb_params,
    n_iter=50,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1,
)

xgb_search.fit(X_train, y_train)

print(f'Best XGBoost params: {xgb_search.best_params_}')
print(f'Best XGBoost RMSE: {-xgb_search.best_score_:.4f}')

best_xgb = xgb_search.best_estimator_

The tuned XGBoost model is the superior solution for this dataset, achieving significantly lower $RMSE$ (0.17) compared to linear models ($RMSE \approx 0.73$). The near-perfect metrics suggest that while the data is likely synthetic, the models successfully identified the underlying deterministic patterns.

In [ ]:
# COMPARISON
feature_names = X_train.columns
fig, axes = plt.subplots(4, 1, figsize=(24, 32))

# 1. LINEAR REGRESSION
df_lr = pd.DataFrame({'Feature': feature_names, 'Value': lr_model.coef_})
df_lr = df_lr.reindex(df_lr['Value'].abs().sort_values(ascending=False).index).head(10)

sns.barplot(
    data=df_lr, x='Value', y='Feature', ax=axes[0], hue='Feature', palette='coolwarm'
)
axes[0].set_title(
    'Baseline Linear Regression (No Regularization)', fontsize=16, fontweight='bold'
)
axes[0].axvline(0, color='black', linestyle='--', linewidth=1)
axes[0].set_xlabel('Coefficient Value')

# 2. RIDGE
df_ridge = pd.DataFrame({'Feature': feature_names, 'Value': best_ridge.coef_})
df_ridge = df_ridge.reindex(
    df_ridge['Value'].abs().sort_values(ascending=False).index
).head(10)

sns.barplot(data=df_ridge, x='Value', y='Feature', ax=axes[1], hue='Feature')
axes[1].set_title('Tuned Ridge', fontsize=14, fontweight='bold')
axes[1].axvline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Importance')

# 3. LASSO
df_lasso = pd.DataFrame({'Feature': feature_names, 'Value': best_lasso.coef_})
df_lasso = df_lasso.reindex(
    df_lasso['Value'].abs().sort_values(ascending=False).index
).head(10)

sns.barplot(data=df_lasso, x='Value', y='Feature', ax=axes[2], hue='Feature')
axes[2].set_title('Tuned Lasso', fontsize=14, fontweight='bold')
axes[2].axvline(0, color='black', linestyle='--', linewidth=1)
axes[2].set_xlabel('Importance')

# 4. XGBOOST
df_xgb = pd.DataFrame(
    {'Feature': feature_names, 'Value': best_xgb.feature_importances_}
)
df_xgb = df_xgb.sort_values(by='Value', ascending=False).head(12)

sns.barplot(data=df_xgb, x='Value', y='Feature', ax=axes[3], hue='Feature')
axes[3].set_title('Tuned XGBoost', fontsize=14, fontweight='bold')
axes[3].set_xlabel('Feature Importance')

plt.tight_layout()
plt.show()

The stability of the models is proven by the consistent ranking of features across Ridge, Lasso, and XGBoost, which all identified family history and age as primary predictors. Additionally, the logical direction of coefficients (e.g., negative for physical activity and positive for BMI) remains stable across the regularized models. While Ridge and Lasso achieved high accuracy ($R^2 \approx 0.99$), their optimal regularization parameters were very low, making them perform similarly to standard regression. These minimal optimal alpha values indicate that the dataset has an exceptionally high signal-to-noise ratio and a nearly deterministic structure, where the relationships between variables are so clear that the models require almost no penalty to prevent overfitting.

In [ ]:
# FINAL TEST
final_models = {
    'Linear Regression': lr_model,
    'Tuned Ridge': best_ridge,
    'Tuned Lasso': best_lasso,
    'Tuned XGBoost': best_xgb,
}

results_comparison = []

for name, model in final_models.items():
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results_comparison.append({'Model': name, 'RMSE': rmse, 'MAE': mae, 'R2 Score': r2})

comparison_df = pd.DataFrame(results_comparison).sort_values(by='RMSE')

print(comparison_df)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

sns.barplot(
    x='RMSE',
    y='Model',
    data=comparison_df,
    palette='viridis',
    ax=axes[0],
    hue='Model',
    legend=False,
)
axes[0].set_title('RMSE')
axes[0].set_xlabel('Root Mean Squared Error')

sns.barplot(
    x='R2 Score',
    y='Model',
    data=comparison_df,
    palette='magma',
    ax=axes[1],
    hue='Model',
    legend=False,
)
axes[1].set_title('R2 Score')
axes[1].set_xlabel('R2 Score')
axes[1].set_xlim(comparison_df['R2 Score'].min() - 0.01, 1.005)

plt.tight_layout()
plt.show()

top_features = [
    'family_history_diabetes',
    'age',
    'physical_activity_minutes_per_week',
    'bmi',
    'glucose_fasting',
]

The tuned XGBoost model achieved the best performance with an R2 of 0.9988, significantly outperforming linear models by capturing complex non-linear relationships.

In [ ]:
# RESIDUALS ANALYSIS
y_pred_final = final_models['Tuned XGBoost'].predict(X_test)
residuals = y_test - y_pred_final

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=residuals, alpha=0.5, color='purple')
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Real data')
plt.ylabel('Residuals')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, color='purple', bins=30)
plt.title('Errors distribution')
plt.xlabel('Errors')
plt.show()

The residual plot shows that the errors are randomly distributed around zero, which confirms the model captured the data patterns correctly. The normal distribution of errors proves that the model's predictions are unbiased and highly accurate. This final analysis validates the tuned XGBoost model as the most reliable choice for predicting diabetes risk.

## Summary

The data was cleaned and preprocessed to address multicollinearity, confirming family history and age as the strongest diabetes risk factors. Using Lasso and Ridge regularization successfully stabilized feature coefficients and improved model interpretability without sacrificing accuracy. The tuned XGBoost model proved to be the most effective solution, achieving an R2 of 0.9988 and a highly precise RMSE of 0.1773.

The near-perfect metrics suggest that the dataset is highly structured and likely synthetic. In a clinical setting, such accuracy is rare due to biological noise. However, for the purpose of this project, the data serves as an excellent benchmark to demonstrate the effectiveness of hyperparameter tuning and the removal of multicollinearity.